# For Making Reference

In [7]:
import cv2
import pyrealsense2

from realsense_aligned_depth import *  #note: I have changed the frame size to 848X480 from 640X480



def empty(a):
    pass

#for displaying trackbar to adjust values of kernal,threshold and min area of contour
cv2.namedWindow("Parameter")
cv2.resizeWindow("Parameter",640,240)
cv2.createTrackbar("kernel","Parameter",7,50,empty)
cv2.createTrackbar("Threshold","Parameter",75,254,empty)
cv2.createTrackbar("AREA","Parameter",250,9000,empty)

# Initialize Camera Intel Realsense
dc = DepthCamera()

#capturing backgroung image
first_frame= True

while first_frame is True:
    ret, depth_bg_frame, color_bg_frame = dc.get_frame()
    bg_bilateral = cv2.bilateralFilter(color_bg_frame, 15, 75, 75)
    bg_gray = cv2.cvtColor(bg_bilateral, cv2.COLOR_BGR2GRAY)
    cv2.imshow("showing Video for bg",bg_gray)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        ff=bg_gray
        #cv2.imwrite("c1.png",bg_gray)
        first_frame= False





# Create mouse event
#cv2.namedWindow("Color frame")
#cv2.setMouseCallback("Color frame", show_distance)

while True:
    ret, depth_frame, color_frame = dc.get_frame()
    if ret is True:
        bilateral = cv2.bilateralFilter(color_frame, 15, 60, 60)
        gray = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY)
        image_sub = cv2.absdiff(ff, gray)
        #kernel=cv2.getTrackbarPos("kernel", "Parameter")#for trackbar
        #if (kernel%2) ==0:
        #    kernel=kernel+1
        #else:
        #    pass
        #blur=cv2.medianBlur(gray,kernel)
        thres=cv2.getTrackbarPos("Threshold","Parameter")#for trackbar
        retu, th1 = cv2.threshold(image_sub,thres,255,cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(color_frame, contours, -1, (0,255,0), 1)
        for i in range (len(contours)):
            cnt2 = contours[i]
            area=cv2.contourArea(cnt2)
            AREA = cv2.getTrackbarPos("AREA", "Parameter")#for trackbar
            if area>=AREA:
                #epsilon = 0.1*cv2.arcLength(cnt2,True)
                #approx = cv2.approxPolyDP(cnt2,epsilon,True)
                rect = cv2.minAreaRect(cnt2) #returns [rectangle center(x, y), (width, height), rotation angle)].
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                color_frame = cv2.drawContours(color_frame,[box],0,(0,255,255),2)
                #M = cv2.moments(cnt2)
                #cx2 = int(M['m10']/M['m00'])
                #cy2 = int(M['m01']/M['m00'])
                cx2=int(rect[0][0])
                cy2=int(rect[0][1])
                point = (cx2 , cy2)
                
                width= int(rect[1][0])
                height= int(rect[1][1])
                
                 # Show distance for a specific point
                cv2.circle(color_frame, point, 4, (0, 0, 255))
                distance =depth_frame[int(point[1]),int(point[0])] #depth_frame[240,320] #
                
                dist=(distance)/10          #input in cm
                cv2.putText(color_frame, "{}cm".format(dist), (point[0], point[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        
                #cvtfac=0.00304535*distance**(0.921814) -0.17026#1#0.00167228*distance -0.00381802
                cvtfac =1#0.00325729*(distance)**(0.914301)-0.144273
                #Rx=(round(width*cvtfac,2))
                #H8
                #Ry=(round(height*cvtfac,2))
                #print(width,height)
                
                #Pixel_to_CM
                
                Rx=(round(cx2*cvtfac,2))
                Ry=(round(cy2*cvtfac,2))
                
                
                #color_frame=cv2.circle(color_frame, (cx2, cy2), 7, (0, 100, 200), -1)
                angle=rect[-1]
                #print(angle)
                if angle<-45:
                    angle=-(90+angle)
                else:
                    angle=-angle
                #print(angle,"deg")
                x_dis=abs(cx2-box[0,0]) #dist btw x value of center of contour and x-value of first vertex of contour
                y_dis=abs(cy2-box[0,1])#dist btw y value of center of contour and y-value of first vertex of contour
                
                ###########################################
                #below values are not used because we change the way to detect orientation
                #a=np.array((box[0,0],box[0,1]))
                #b=np.array((box[1,0],box[1,1]))
                #c=np.array((box[2,0],box[2,1]))
                #ab=np.linalg.norm(a-b)
                #bc=np.linalg.norm(b-c)
                #print(ab,bc)
                
                #if (ab<bc and (angle<=0 and angle>-10)) or (ab>bc and (angle>0 and angle<10)):
                 #   frame=cv2.putText(frame,"Vertical" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    #print("its verticle")
                #elif (ab>bc and (angle<=0 and angle>-10)) or (ab<bc and (angle>0 and angle<10)):
                    #print("its Horizontal")
                 #   frame=cv2.putText(frame,"Horizontal" , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                #############################################
                
               
                if (x_dis<y_dis and (angle<10 and angle>-10)):
                    color_frame=cv2.putText(color_frame,"v"+str( Rx)+","+str( Ry) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    #print("its verticle")
                elif (x_dis>y_dis and (angle<10 and angle>-10)):
                    #print("its Horizontal")
                    color_frame=cv2.putText(color_frame,"H"+str(Rx)+","+str(Ry), (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                else:
                    color_frame=cv2.putText(color_frame,"I"+str(Rx)+","+str(Ry) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (155, 0, 0), 2)
                    #print("Its inclined")

        
    else:
        break

    #cv2.imshow("depth frame", depth_frame)
    cv2.imshow("Color frame", color_frame)
    #cv2.imshow("gray frame", gray)
    #cv2.imshow("blur frame", blur)
    #cv2.imshow("th1 frame", th1)
    key = cv2.waitKey(1)
    if key == 27:
        dc.release()
        break
cv2.destroyAllWindows()

Package           Version
----------------- -----------
backcall          0.2.0
colorama          0.4.4
debugpy           1.5.1
decorator         5.1.0
entrypoints       0.3
ipykernel         6.6.0
ipython           7.30.1
jedi              0.18.1
jupyter-client    7.1.0
jupyter-core      4.9.1
keyboard          0.13.5
matplotlib-inline 0.1.3
nest-asyncio      1.5.4
numpy             1.21.5
opencv-python     4.5.4.60
parso             0.8.3
pickleshare       0.7.5
pip               21.1.3
prompt-toolkit    3.0.24
Pygments          2.10.0
pylogix           0.7.16
pyrealsense2      2.50.0.3812
python-dateutil   2.8.2
pywin32           303
pyzmq             22.3.0
setuptools        56.0.0
six               1.16.0
tornado           6.1
traitlets         5.1.1
wcwidth           0.2.5
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


# For Depalletizer Application

In [1]:
import cv2
import pyrealsense2
import numpy as np
from pylogix import PLC
import keyboard
import time
from realsense_aligned_depth import *

def Sort(s):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    s.sort(key = lambda x: x[0])
    return s
print("THE SYSTEM IS ON _ press")

while True:
    
    comm =PLC()
    comm.IPAddress = '192.168.255.2'
    comm.Write('signal_to_rob_02',0)
    comm.Write('signal_to_rob_04',0)
    
    comm.Write('signal_to_rob_06',0)
    no_carton=0
    print("waiting for robot to go to home position")

    while True: #should be true only in the presence of pallet
        sig_1=comm.Read('signal_to_pc_03')
        
        # if keyboard.is_pressed('p'):
        #     break
        if sig_1.Value is True:
            break
    print("Reached home position robot waiting")
    
    
    comm.Write('signal_to_rob_05',0)
    
    
    #initialise the depth camera
    dc = DepthCamera()
    first_frame= True
    list=[]
    np.array(list)
    while first_frame is True:
        ret,depth_frame,color_frame= dc.get_frame()
        sval=60000
        for i in range(50,400):
            for j in range(125,700):
                if sval>depth_frame[i][j] & depth_frame[i][j]!=0:
                    sval=depth_frame[i][j]
        if len(list)>10:
            list.pop(0)
            list.append(sval)
        else:
            list.append(sval)            
            
        cv2.imshow("color_frame", color_frame)
        #cv2.imshow("depth_frame", depth_frame)
    
        if cv2.waitKey(1) & len(list)==5: #0xFF == ord('c'):

            
            list.pop(0)
            avg_sval= sum(list)/(len(list))
            #ff_d=depth_frame
            #ff_c=color_frame
            first_frame= False
            print(list)
            print(len(list))
            print(avg_sval)
        key= cv2.waitKey(1)
        if key == 27:
            break
    if avg_sval>1050:
        diff= avg_sval-1050
        move_x = 50
        move_y = -70
        move_z = diff-1480
        
    else:
        diff=avg_sval-1050
        move_x= 50#70
        move_y = -70#-18
        move_z=-1480+diff

    comm.Write('x_axis_frm_pc', move_y)
    comm.Write('y_axis_frm_pc', move_x)
    comm.Write('z_axis_frm_pc', move_z)
    sig_2=1
    comm.Write('signal_to_rob_02',sig_2)
    print("moving robot in z direction :", move_z)
    while True:
        #movement of robot in z direction
        sig_3=comm.Read("signal_to_pc_01")
        if sig_3.Value is True:
            break
    sig_2=0
    comm.Write('signal_to_rob_02',sig_2)
    print("Moved downwards, now waiting for carton detection")
    first_frame= True
    list_2=[]
    np.array(list_2)
    while first_frame is True:
        ret,depth_frame,color_frame= dc.get_frame()
        sval=60000
        for i in range(50,400):
            for j in range(125,700):
                if sval>depth_frame[i][j] & depth_frame[i][j]!=0:
                    sval=depth_frame[i][j]
        if len(list_2)>50:
            list_2.pop(0)
            list_2.append(sval)
        else:
            list_2.append(sval)            
            
        cv2.imshow("color_frame", color_frame)
        #cv2.imshow("depth_frame", depth_frame)
    
        if cv2.waitKey(1) & len(list_2)==15: #0xFF == ord('c'):

            
            list_2.pop(0)
            avg_sval= sum(list_2)/(len(list_2))
            ff_d=depth_frame
            ff_c=color_frame
            print(list_2)
            print(len(list_2))
            print(avg_sval)

            if avg_sval<=1180:
                add_value=80
            elif avg_sval< 1320:
                add_value=80
            elif avg_sval< 1480:
                add_value= 80
            elif avg_sval> 1480:
                add_value=100

            # Getting the depth sensor's depth scale (see rs-align example for explanation)
            #depth_sensor = dc.get_device().first_depth_sensor()
            depth_scale = 0.0010000000474974513 #depth_sensor.get_depth_scale()
            # We will be removing the background of objects more than
            #  clipping_distance_in_meters meters away
            clipping_distance_in_meters = (avg_sval+add_value)/1000 #1 meter
            clipping_distance = clipping_distance_in_meters / depth_scale

            # Remove background - Set pixels further than clipping_distance to grey
            grey_color = 0
            depth_image_3d = np.dstack((ff_d,ff_d,ff_d)) #depth image is 1 channel, color is 3 channels
            bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, ff_c)

            """
            Start of:
            Reading input image
            """

            # Reading image with OpenCV library
            # In this way image is opened already as numpy array
            # WARNING! OpenCV by default reads images in BGR format
            # Pay attention! If you're using Windows, the path might looks like:
            # r'images\woman-working-in-the-office.jpg'
            # or:
            # 'images\\woman-working-in-the-office.jpg'
            #image_BGR = cv2.imread('images/woman-working-in-the-office.jpg') #for linux
            #image_BGR = cv2.imread(r'K:\Vision Tech\YOLO\Udemy YOLO\YOLO-3-OpenCV\YOLO-3-OpenCV\images\test_carton2.jpg') #for windows
            image_BGR=bg_removed
            # Showing Original Image
            # Giving name to the window with Original Image
            # And specifying that window is resizable
            ###############################cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
            # Pay attention! 'cv2.imshow' takes images in BGR format
            ################################cv2.imshow('Original Image', image_BGR)
            # Waiting for any key being pressed
            ###################################|cv2.waitKey(0)
            # Destroying opened window with name 'Original Image'
            ######################################cv2.destroyWindow('Original Image')

            # # Check point
            # # Showing image shape
            # print('Image shape:', image_BGR.shape)  # tuple of (511, 767, 3)

            # Getting spatial dimension of input image
            h, w = image_BGR.shape[:2]  # Slicing from tuple only first two elements

            # # Check point
            # # Showing height an width of image
            # print('Image height={0} and width={1}'.format(h, w))  # 511 767

            """
            End of: 
            Reading input image
            """


            """
            Start of:
            Getting blob from input image
            """

            # Getting blob from input image
            # The 'cv2.dnn.blobFromImage' function returns 4-dimensional blob
            # from input image after mean subtraction, normalizing, and RB channels swapping
            # Resulted shape has number of images, number of channels, width and height
            # E.G.:
            # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
            blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                                        swapRB=True, crop=False)

            # # Check point
            # print('Image shape:', image_BGR.shape)  # (511, 767, 3)
            # print('Blob shape:', blob.shape)  # (1, 3, 416, 416)

            # # Check point
            # # Showing blob image in OpenCV window
            # # Slicing blob image and transposing to make channels come at the end
            # blob_to_show = blob[0, :, :, :].transpose(1, 2, 0)
            # print(blob_to_show.shape)  # (416, 416, 3)
            #
            # # Showing Blob Image
            # # Giving name to the window with Blob Image
            # # And specifying that window is resizable
            # cv2.namedWindow('Blob Image', cv2.WINDOW_NORMAL)
            # # Pay attention! 'cv2.imshow' takes images in BGR format
            # # Consequently, we DO need to convert image from RGB to BGR firstly
            # # Because we have our blob in RGB format
            # cv2.imshow('Blob Image', cv2.cvtColor(blob_to_show, cv2.COLOR_RGB2BGR))
            # # Waiting for any key being pressed
            # cv2.waitKey(0)
            # # Destroying opened window with name 'Blob Image'
            # cv2.destroyWindow('Blob Image')

            """
            End of:
            Getting blob from input image
            """


            """
            Start of:
            Loading YOLO v3 network
            """

            # Loading COCO class labels from file
            # Opening file
            # Pay attention! If you're using Windows, yours path might looks like:
            # r'yolo-coco-data\coco.names'
            # or:
            # 'yolo-coco-data\\coco.names'
            #with open('yolo-coco-data/coco.names') as f:#for linux
            with open(r'K:\Vision Tech\YOLO\Udemy YOLO\YOLO-3-OpenCV\YOLO-3-OpenCV\carton.names') as f: #for windows
                # Getting labels reading every line
                # and putting them into the list
                labels = [line.strip() for line in f]


            # # Check point
            # print('List with labels names:')
            # print(labels)

            # Loading trained YOLO v3 Objects Detector
            # with the help of 'dnn' library from OpenCV
            # Pay attention! If you're using Windows, yours paths might look like:
            # r'yolo-coco-data\yolov3.cfg'
            # r'yolo-coco-data\yolov3.weights'
            # or:
            # 'yolo-coco-data\\yolov3.cfg'
            # 'yolo-coco-data\\yolov3.weights'
            # network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
            #                                      'yolo-coco-data/yolov3.weights') #for linux
            network = cv2.dnn.readNetFromDarknet(r'K:\Vision Tech\YOLO\Udemy YOLO\YOLO-3-OpenCV\YOLO-3-OpenCV\yolo-coco-data\yolov4-carton-test.cfg',
                                                r'K:\Vision Tech\YOLO\Udemy YOLO\YOLO-3-OpenCV\YOLO-3-OpenCV\yolo-coco-data\yolov4-carton-train_last.weights') #for windows

            # Getting list with names of all layers from YOLO v3 network
            layers_names_all = network.getLayerNames()

            # # Check point
            # print()
            # print(layers_names_all)

            # Getting only output layers' names that we need from YOLO v3 algorithm
            # with function that returns indexes of layers with unconnected outputs
            layers_names_output = \
                [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

            # # Check point
            # print()
            # print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

            # Setting minimum probability to eliminate weak predictions
            probability_minimum = 0.2

            # Setting threshold for filtering weak bounding boxes
            # with non-maximum suppression
            threshold = 0.3

            # Generating colours for representing every detected object
            # with function randint(low, high=None, size=None, dtype='l')
            colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

            # # Check point
            # print()
            # print(type(colours))  # <class 'numpy.ndarray'>
            # print(colours.shape)  # (80, 3)
            # print(colours[0])  # [172  10 127]

            """
            End of:
            Loading YOLO v3 network
            """


            """
            Start of:
            Implementing Forward pass
            """

            # Implementing forward pass with our blob and only through output layers
            # Calculating at the same time, needed time for forward pass
            network.setInput(blob)  # setting blob as input to the network
            start = time.time()
            output_from_network = network.forward(layers_names_output)
            end = time.time()

            # Showing spent time for forward pass
            print('Objects Detection took {:.5f} seconds'.format(end - start))

            """
            End of:
            Implementing Forward pass
            """


            """
            Start of:
            Getting bounding boxes
            """

            # Preparing lists for detected bounding boxes,
            # obtained confidences and class's number
            bounding_boxes = []
            confidences = []
            class_numbers = []


            # Going through all output layers after feed forward pass
            for result in output_from_network:
                # Going through all detections from current output layer
                for detected_objects in result:
                    # Getting 80 classes' probabilities for current detected object
                    scores = detected_objects[5:]
                    # Getting index of the class with the maximum value of probability
                    class_current = np.argmax(scores)
                    # Getting value of probability for defined class
                    confidence_current = scores[class_current]

                    # # Check point
                    # # Every 'detected_objects' numpy array has first 4 numbers with
                    # # bounding box coordinates and rest 80 with probabilities for every class
                    # print(detected_objects.shape)  # (85,)

                    # Eliminating weak predictions with minimum probability
                    if confidence_current > probability_minimum:
                        # Scaling bounding box coordinates to the initial image size
                        # YOLO data format keeps coordinates for center of bounding box
                        # and its current width and height
                        # That is why we can just multiply them elementwise
                        # to the width and height
                        # of the original image and in this way get coordinates for center
                        # of bounding box, its width and height for original image
                        box_current = detected_objects[0:4] * np.array([w, h, w, h])

                        # Now, from YOLO data format, we can get top left corner coordinates
                        # that are x_min and y_min
                        x_center, y_center, box_width, box_height = box_current
                        x_min = int(x_center - (box_width / 2))
                        y_min = int(y_center - (box_height / 2))

                        # Adding results into prepared lists
                        bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                        confidences.append(float(confidence_current))
                        class_numbers.append(class_current)

            """
            End of:
            Getting bounding boxes
            """


            """
            Start of:
            Non-maximum suppression
            """

            # Implementing non-maximum suppression of given bounding boxes
            # With this technique we exclude some of bounding boxes if their
            # corresponding confidences are low or there is another
            # bounding box for this region with higher confidence

            # It is needed to make sure that data type of the boxes is 'int'
            # and data type of the confidences is 'float'
            # https://github.com/opencv/opencv/issues/12789
            results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                                    probability_minimum, threshold)

            """
            End of:
            Non-maximum suppression
            """


            """
            Start of:
            Drawing bounding boxes and labels
            """

            # Defining counter for detected objects
            counter = 1
            arr=[]
            # Checking if there is at least one detected object after non-maximum suppression
            if len(results) > 0:
                # Going through indexes of results
                for i in results.flatten():
                    # Showing labels of the detected objects
                    print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))

                    # Incrementing counter
                    counter += 1

                    # Getting current bounding box coordinates,
                    # its width and height
                    x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
                    box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
                    cx1= x_min+(box_width/2)
                    cy1= y_min+(box_height/2)
                    point = (cx1 , cy1)

                    distance = avg_sval #ff_d[int(point[1]),int(point[0])] #avg_sval ##nf_depth[240, 320] # [point[1],point[0]]

                    cv2.circle(image_BGR, (int(cx1) , int(cy1)), 4, (0, 0, 255))

                    dist=distance/10          #input in cm
                    each_carton_dist=ff_d[int(point[1]),int(point[0])]
                    cv2.putText(image_BGR, "{}cm".format(each_carton_dist), (int(point[0]), int(point[1] - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

                    cvtfac=0.00325729*distance**(0.914301)-0.144273#0.00304535*distance**(0.921814) -0.17026 #0.00167228*distance -0.00381802
                    cx2=(round(cx1*cvtfac,2))
                    cy2=(round(cy1*cvtfac,2))


                    if (box_width>=box_height):
                        cv2.putText(image_BGR,"H"+str(cx2)+","+str(cy2), (int(cx1-10), int(cy1-10)), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                        orientation=0
                    else:
                        cv2.putText(image_BGR,"V"+str(cx2)+","+str(cy2), (int(cx1-10), int(cy1-10)), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                        orientation=900

                    #To ROBOT
                    robx=(round((cx1-424)*cvtfac,0))
                    roby=(round((240-cy1)*cvtfac,0))

                    arr.append([robx,roby, move_z-30+distance,0,0 ,orientation])
                    # Preparing colour for current bounding box
                    # and converting from numpy array to list
                    colour_box_current = colours[class_numbers[i]].tolist()

                    # # # Check point
                    # print(type(colour_box_current))  # <class 'list'>
                    # print(colour_box_current)  # [172 , 10, 127]

                    # Drawing bounding box on the original image
                    cv2.rectangle(image_BGR, (x_min, y_min),
                                (x_min + box_width, y_min + box_height),
                                colour_box_current, 2)

                    # Preparing text with label and confidence for current bounding box
                    text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                        confidences[i])

                    # Putting text with label and confidence on the original image
                    cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                                cv2.FONT_HERSHEY_COMPLEX, 1, colour_box_current, 2)

                    # Putting text with center location on the original image
                    # cv2.putText(image_BGR, "{}cm".format(x_min,box_width), (x_center, y_center),
                    #             cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
            else:
                print("NO CARTONS DETECTED")
                sig_4=1
                #comm.Write('signal_to_rob_04',sig_4)
                sig_6=1
                comm.Write('signal_to_rob_06',sig_6)
                no_carton=1
                break



            # Comparing how many objects where before non-maximum suppression
            # and left after
            print(arr)
            print()
            print('Total objects been detected:', len(bounding_boxes))
            print('Number of objects left after non-maximum suppression:', counter - 1)

            """
            End of:
            Drawing bounding boxes and labels
            """


            # Showing Original Image with Detected Objects
            # Giving name to the window with Original Image
            # And specifying that window is resizable
            cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
            # Pay attention! 'cv2.imshow' takes images in BGR format
            cv2.imshow('Detections', image_BGR)
            # Waiting for any key being pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                #Destroying opened window with name 'Detections'
                cv2.destroyWindow('Detections')
            


            """
            Some comments

            With OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional
            so called 'blob' from input image after mean subtraction,
            normalizing, and RB channels swapping. Resulted shape has:
            - number of images
            - number of channels
            - width
            - height
            E.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
            """

            #comm.Write('signal_to_rob_05',0)

            arr=Sort(arr)
            print(arr)
            for i in range(len(arr)):
                comm =PLC()
                comm.IPAddress = '192.168.255.2'
                n=comm.Read('pulse_to_pc')
                print(n.Value)
                real_x=arr[i][0]
                real_y=arr[i][1]
                real_z=arr[i][2]
                real_rx=arr[i][3]
                real_ry=arr[i][4]
                real_rz=arr[i][5]
                print("Grab item from x:" +str(real_y)+", y:"+str(real_x)+", z:"+str(real_z)+", Rz:"+str(real_rz))
                comm.IPAddress = '192.168.255.2'
                comm.Write('x_axis_frm_pc', real_y)
                comm.Write('y_axis_frm_pc', real_x)
                comm.Write('z_axis_frm_pc', real_z)
                comm.Write('r_x_frm_pc', real_rx)
                comm.Write('r_y_frm_pc', real_ry)
                comm.Write('r_z_frm_pc', real_rz)
                sig_4=1
                comm.Write('signal_to_rob_04',sig_4)#
                comm.Close()
                #time.sleep(20)
                while n.Value is False:
                    #print(n.Value)
                    n=comm.Read('pulse_to_pc')# should get pulse of '1'
                    #print(n.Value)
                    if keyboard.is_pressed('p'):        # press 0 to exit the while loop
                        arr=[]
                        break
                if not arr:
                    break
            sig_5=1
            comm.Write('signal_to_rob_05',sig_5)
            first_frame=False
    


    print("exited inner loop")
    if no_carton==1 or keyboard.is_pressed('q'):
        print("Exiting the code")
        #sig_6=1
        #comm.Write('signal_to_rob_06',sig_6)
        cv2.destroyAllWindows()
        dc.release()
        break


THE SYSTEM IS ON _ press
waiting for robot to go to home position
Reached home position robot waiting
[962, 959, 963, 967]
4
962.75
moving robot in z direction : -1567.25
Moved downwards, now waiting for carton detection
[1045, 1049, 1048, 1045, 1053, 1040, 1049, 1042, 1046, 1023, 1048, 1045, 1035, 1053]
14
1044.357142857143
Objects Detection took 0.50087 seconds
Object 1: carton
Object 2: carton
Object 3: carton
Object 4: carton
Object 5: carton
Object 6: carton
[[-248.0, 153.0, -552.8928571428571, 0, 0, 0], [72.0, 116.0, -552.8928571428571, 0, 0, 900], [-215.0, -174.0, -552.8928571428571, 0, 0, 900], [97.0, -198.0, -552.8928571428571, 0, 0, 0], [407.0, 138.0, -552.8928571428571, 0, 0, 0], [432.0, -182.0, -552.8928571428571, 0, 0, 900]]

Total objects been detected: 45
Number of objects left after non-maximum suppression: 6
[[-248.0, 153.0, -552.8928571428571, 0, 0, 0], [-215.0, -174.0, -552.8928571428571, 0, 0, 900], [72.0, 116.0, -552.8928571428571, 0, 0, 900], [97.0, -198.0, -552.8

In [9]:
comm.Write('signal_to_rob_05',1)


Response(TagName=signal_to_rob_05, Value=1, Status=Success)

In [3]:
a=comm.Read('signal_to_rob_05')
print(a.Value)

True


In [6]:
pip install --upgrade opencv-python==4.5.3.56

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.62
    Uninstalling opencv-python-4.5.5.62:
      Successfully uninstalled opencv-python-4.5.5.62
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
pip uninstall opencv-python

^C
Note: you may need to restart the kernel to use updated packages.
